In [1]:
!pip install curl_cffi pandas beautifulsoup4

import pandas as pd
from curl_cffi import requests
from bs4 import BeautifulSoup
import urllib.parse
import time
import random

# Membaca dataset awal
df_source = pd.read_csv("C:\TUGAS\pemteks\dataset_lagu_final (3).csv")

headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'id,id-ID;q=0.9,en-US;q=0.8,en;q=0.7,ms;q=0.6',
    'cache-control': 'no-cache',
    'pragma': 'no-cache',
    'priority': 'u=1, i',
    'referer': 'https://genius.com/search?q=indonesia',
    'sec-ch-ua': '"Chromium";v="124", "Google Chrome";v="124", "Not_A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest'
}

endpoint_lists = "https://genius.com/api/search/multi"

print("--- Memulai Pencarian Metadata & Perbaikan Tahun ---")

for index, row in df_source.iterrows():
    title = row['Judul']
    # Pastikan kolom artis ada, jika error ganti 'Artis' dengan nama kolom yang sesuai di csvmu
    artist = row['Artis'] 
    search_query = f"{title} {artist}"
    
    params = {
        'q': search_query
    }

    query_encoded = urllib.parse.quote(search_query)
    headers['referer'] = "https://genius.com/search?q=" + query_encoded

    try:
        response = requests.get(endpoint_lists, params=params, headers=headers, impersonate="chrome124")
        response.raise_for_status()

        data = response.json()
        sections = data["response"]["sections"]

        song = None
        for section in sections:
            if section["type"] == "song":
                if len(section["hits"]) > 0:
                    song = section["hits"][0]["result"]
                    break

        if song is None:
            print(f"Lagu {title} - {artist} tidak ditemukan di Genius")
            continue

        # Simpan metadata Genius
        df_source.loc[index, "Genius ID"] = song["id"]
        df_source.loc[index, "Genius Path"] = song["path"]
        df_source.loc[index, "Api Path"] = song["api_path"]
        df_source.loc[index, "Header Image"] = song["header_image_url"]
        
        # LOGIKA PERBAIKAN TAHUN (FIX YEAR)
        # Prioritas 1: Ambil dari komponen tanggal (paling akurat)
        genius_year = None
        if song.get("release_date_components") and song["release_date_components"].get("year"):
            genius_year = song["release_date_components"]["year"]
        
        # Prioritas 2: Parsing dari string display jika komponen null
        elif song.get("release_date_for_display"):
            # Biasanya format "Month DD, YYYY" atau "YYYY"
            date_str = str(song["release_date_for_display"])
            # Ambil 4 digit terakhir sebagai tahun
            if len(date_str) >= 4 and date_str[-4:].isdigit():
                genius_year = int(date_str[-4:])

        # Update kolom Tahun jika data ditemukan
        if genius_year:
            df_source.loc[index, "Tahun"] = genius_year
            print(f"Metadata & Tahun ditemukan: {title} ({genius_year})")
        else:
            print(f"Metadata ditemukan (Tanpa Tahun): {title}")

        time.sleep(random.uniform(1, 2))

    except requests.exceptions.RequestException as e:
        print(f"Error pencarian {title}: {e}")

print("--- Memulai Scraping Lirik ---")

for index, row in df_source.iterrows():
    path = row['Genius Path']
    
    if pd.isna(path):
        continue

    try:
        url = f"https://genius.com{path}"
        response = requests.get(url, impersonate="chrome124", headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, "html.parser")
        lyrics_containers = soup.select('div[data-lyrics-container="true"]')
        
        full_lyrics = []
        for container in lyrics_containers:
            for br in container.find_all("br"):
                br.replace_with("\n")
            text = container.get_text()
            full_lyrics.append(text)

        if full_lyrics:
            clean_lyrics = "\n\n".join(full_lyrics)
            # Membersihkan suffix " Lyrics" yang kadang muncul otomatis
            if ' Lyrics' in clean_lyrics:
                 clean_lyrics = clean_lyrics.split(' Lyrics')[1]
            
            df_source.loc[index, 'Lirik'] = clean_lyrics
            print(f"Lirik diambil: {row['Judul']}")
        else:
            print(f"Lirik kosong/instrumental: {row['Judul']}")

        time.sleep(2)

    except Exception as e:
        print(f"Error ambil lirik {row['Judul']}: {e}")

# Ubah newline jadi tag khusus agar rapi di CSV
if 'Lirik' in df_source.columns:
    df_source['Lirik'] = df_source['Lirik'].str.replace('\n', ':newline:')

# Simpan ke file baru
df_source.to_csv("dataset_lagu_lengkap_final_fixed_year.csv", index=False)
print("Proses selesai. File disimpan sebagai dataset_lagu_lengkap_final_fixed_year.csv")

--- Memulai Pencarian Metadata & Perbaikan Tahun ---
Metadata & Tahun ditemukan: Dan... (1999)
Metadata & Tahun ditemukan: Aku Milikmu (1994)
Metadata & Tahun ditemukan: Matahariku (1991)
Metadata & Tahun ditemukan: Memang (2000)
Metadata & Tahun ditemukan: Sobat (1999)
Metadata & Tahun ditemukan: Terbunuh Sepi (1994)
Metadata & Tahun ditemukan: Bebas (1994)
Metadata ditemukan (Tanpa Tahun): Tak Bisa Ke Lain Hati
Metadata & Tahun ditemukan: Mahadewi (1999)
Metadata ditemukan (Tanpa Tahun): Yogyakarta
Metadata ditemukan (Tanpa Tahun): Kirana
Metadata ditemukan (Tanpa Tahun): Cerita Cinta
Metadata & Tahun ditemukan: Kau Yang Terindah (1994)
Metadata ditemukan (Tanpa Tahun): Untukku
Metadata & Tahun ditemukan: Salah (1997)
Metadata & Tahun ditemukan: Seperti Kekasihku (1999)
Metadata & Tahun ditemukan: Kamu Harus Pulang (1994)
Metadata ditemukan (Tanpa Tahun): Tak Bisa Ke Lain Hati
Metadata ditemukan (Tanpa Tahun): Satu Yang Tak Bisa Lepas
Metadata ditemukan (Tanpa Tahun): Terlanjur Sayan

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22208\147608867.py:126: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[Lirik "Dan..."]

[Verse 1]
Dan...
Dan bila esok, datang kembali
Seperti sedia kala
Dimana kau bisa bercanda

[Verse 2]
Dan...
Perlahan kaupun, lupakan aku
Mimpi burukmu
Dimana t'lah kutancapkan duri tajam
Kaupun menangis, menangis sedih
Maafkan aku

[Verse 3]
Dan...
Bukan maksudku, bukan inginku
Melukaimu sadarkah kau di sini 'ku pun terluka
Melupakanmu, menepikanmu
Maafkan aku

[Chorus]
Lupakanlah saja diriku
Bila itu bisa membuatmu kembali bersinar
Dan berpijar seperti dulu kala
Caci maki saja diriku
Bila itu bisa membuatmu kembali bersinar
Dan berpijar seperti dulu kala




[Verse 3]
Dan...
Bukan maksudku, bukan inginku
Melukaimu sadarkah kau di sini 'ku pun terluka
Melupakanmu, menepikanmu
Maafkan aku

[Chorus]
Lupakanlah saja diriku
Bila itu bisa membuatmu kembali bersinar
Dan berpijar

Lirik diambil: Dan...
Lirik diambil: Aku Milikmu
Lirik diambil: Matahariku
Lirik diambil: Memang
Lirik diambil: Sobat
Lirik diambil: Terbunuh Sepi
Lirik diambil: Bebas
Lirik diambil: Tak Bisa Ke Lain Hati
Lirik diambil: Mahadewi
Lirik diambil: Yogyakarta
Lirik diambil: Kirana
Lirik diambil: Cerita Cinta
Lirik diambil: Kau Yang Terindah
Lirik diambil: Untukku
Lirik diambil: Salah
Lirik diambil: Seperti Kekasihku
Lirik diambil: Kamu Harus Pulang
Lirik diambil: Tak Bisa Ke Lain Hati
Lirik diambil: Satu Yang Tak Bisa Lepas
Lirik diambil: Terlanjur Sayang
Lirik diambil: Begitu Indah
Lirik diambil: Pesawat Tempurku
Lirik diambil: Negeri Di Awan
Lirik diambil: Aku Cinta Padamu
Lirik diambil: Kirana
Lirik diambil: Aku Milikmu
Lirik diambil: Aku Disini Untukmu
Lirik diambil: Kamulah Satu - Satunya
Lirik diambil: Satu Hati (Kita Semestinya)
Lirik diambil: Persembahan Dari Surga
Lirik diambil: Mawar Merah
Lirik diambil: Koepoe Liarkoe
Lirik diambil: Bintang Kehidupan
Lirik diambil: Restoe Boemi
L